In [1]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

from netCDF4 import Dataset
import glob
import matplotlib.pyplot as plt
from skimage.measure import label
from skimage.measure import regionprops
from sklearn.neighbors import NearestNeighbors
from statsmodels.distributions.empirical_distribution import ECDF as ecdf
from scipy.ndimage import label, binary_dilation
from scipy import optimize

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import pickle

import os
    
plotdir = './plots/precip/'
if not os.path.exists(plotdir):
    os.makedirs(plotdir)  
    
#sns.set(font='Franklin Gothic Book',
sns.set(rc={
 'axes.axisbelow': False,
 'axes.edgecolor': 'black',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'black',#dimgrey
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'black',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'xtick.bottom': True,
 'xtick.color': 'black',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'black',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': False})

sns.set_context("notebook", rc={"font.size":16,"axes.titlesize":20,"axes.labelsize":18})


def label_cluster(cluster_mask, stencil=np.ones((1,1)), undilute=True):
    """
    Find coherent clusters.
    
    Method to find and label coherent clusters.
    The stencil can be used to count also masked points
    which are not direct neighbours to the cluster.
    
    Input
    -----
    cluster_mask : array-like
        Two dimensional binary array being True where a cluster
        exists and False otherwise
    
    stencil : np.ones(N,N)
        Kernel/stencil to check connectivities.
        N should be uneven, because otherwise the stencil
        is not centered
    
    undilute : boolean
        If True (default), then only the original cluster_mask
        positions are labeled, otherwise also the labels for the
        diluted fieled are included
    
    Returns
    -------
    cluster_labels : np.array, same shape as cluster_mask
        Array where connected clusters are given one number starting at 1.
        Background is 0.
    
    Example
    -------
    >>> a = np.array([[1., 0.,0, 0., 1.],
       [1., 0., 0, 0., 0.],
       [1., 0., 0, 1., 1.],
       [1., 0, 0, 1., 1.]])
    >>> label_cluster(a,stencil=np.ones((1,1)))
    array([[1, 0, 0, 0, 2],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 3, 3],
       [1, 0, 0, 3, 3]], dtype=int32)
    >>> label_cluster(a,stencil=np.ones((3,3)), undilute=False)
    array([[1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]], dtype=int32)
    >>> label_cluster(a,stencil=np.ones((3,3)))
        np.array([[1, 0, 0, 0, 0, 2],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 2, 2],
       [1, 0, 0, 0, 2, 2]], dtype=int32)
    >>> label_cluster(a, stencil=np.array([[0,1,0],[1,1,1],[0,1,0]]),undilute=False)
    array([[1, 1, 0, 0, 2, 2],
       [1, 1, 0, 0, 2, 2],
       [1, 1, 0, 2, 2, 2],
       [1, 1, 0, 2, 2, 2]], dtype=int32)
    """
    import numpy as np
    from scipy.ndimage import label, binary_dilation
    labels = label(cluster_mask, structure=stencil)[0]

    return labels


def calc_cluster_size(cluster_mask, cluster_labels,normalize=True):
    """
    Calculate the cluster size
    
    Input
    -----
    cluster_mask : array-like
        Two dimensional binary array being True where a cluster
        exists and False otherwise
    
    cluster_labels : np.array, same shape as cluster_mask
        Array where connected clusters are given one number starting at 1.
        Background is 0.
    
    normalize : boolean
        If normalize is True (default) than the cluster size is
        given as a fraction of the domain, otherwise it is
        the count of clustery pixels
    
    Returns
    -------
    cluster_size : float
        Area or fraction of each single cluster
    """
    import numpy as np
    from scipy.ndimage import sum as cluster_sum
    sum = cluster_sum(cluster_mask,cluster_labels,\
                       np.unique(cluster_labels)[1::])
    if normalize:
        sum = sum/(cluster_mask.shape[0]*cluster_mask.shape[1])
    return sum

In [2]:
# THIS STEP IS ONLY NECESSARY IF PICKLE IS NOT USED!!

array_yea= ["2020"]
array_mon= ["01","02","03","04","05","06","07","08"]#


exp_list      = ["4km_fesom","ifs_cycle2_4km_tp","9km_nemo","ifs_cycle2_9km_tp","9km_fesom","ifs_cycle2_3km_tp"]
expid_list= ["hzfy","hqys","hz9o","hr2n","hz9o","hr0n"]
# FESOM family is red, NEMO family blue
color_list    = ["#ff0000","#ff0000","#0000ff","#0000ff","purple","darkorange"]
# give IFS resolution only, FESOM/NEMO don't change from 25/5km
name_list   = ["Cycle 3, 4.4km, IFS-FESOM","Cycle 2, 4.4km, IFS-FESOM","Cycle 3, 9km, IFS-NEMO","Cycle 2, 9km, IFS-NEMO","Cycle 3, 9km, IFS-FESOM","Cycle 2, 2.8km, IFS-FESOM"]
# Cycle 3 is solid, Cycle 2/1 dashed
array_linestyle= ["solid","dashed","solid","dashed","solid","dashed"]
# main simulation is thicker to stand out
array_linewidth= [2,1,1,1,1,1]

#stencil = np.array([[0,1,0],[1,1,1],[0,1,0]])    
stencil = np.array([[1,1,1],[1,1,1],[1,1,1]])

cluster_size_all = {}
cluster_size_all[0] = []
cluster_size_all[1] = []
cluster_size_all[2] = []
cluster_size_all[3] = []
cluster_size_all[4] = []
cluster_size_all[5] = []
cluster_size_GPM_all = []

for (y,YEA) in enumerate(array_yea):
   for (m,MON) in enumerate(array_mon):
        print(MON)

        file=f'/work/bm1235/u233156/observations/GPM_IMERG/GPM_IMERG_hourly_{YEA}_{MON}_tropics.nc'
        Pr = xr.open_dataset(file,engine='netcdf4')['precipitationCal'].load()
        Pr.values = Pr.values
        Pr_99_1_0=np.where(Pr > 3,1,0)
        #num=0
        dummy=[]
        for t in range(len(Pr_99_1_0[:,0,0])):
              cluster_labels = label_cluster(Pr_99_1_0[t,:,:], stencil)
              #num=num+np.amax(cluster_labels)
              dummy.extend(calc_cluster_size(Pr_99_1_0[t,:,:], cluster_labels, normalize=False))
        cluster_size_GPM_all.extend(dummy)
            
        for (a,exp),expid in zip(enumerate(exp_list),expid_list): 
            if  array_linestyle[a] == "solid":  
              file=f'/work/bm1235/u233156/IFS_cycle3/{exp}/{expid}_2D_precip_hourly_remap_0.1x0.1_{YEA}{MON}_tropics.nc'
              Pr = xr.open_dataset(file,engine='netcdf4')['tp'].load()
              Pr.values = Pr.values
            else:
              file=f'/work/bm1235/u233156/IFS_cycle2/tp_mmperd_{expid}_alltimes_remap_0.1x0.1_tropics_{YEA}{MON}.nc'
              Pr = xr.open_dataset(file,engine='netcdf4')['var228'].load()
              Pr.values = Pr.values /24 #from mm/d to mm/h                
            Pr_99_1_0=np.where(Pr > 3,1,0)
            #num=0
            dummy=[]
            for t in range(len(Pr_99_1_0[:,0,0])):
              cluster_labels = label_cluster(Pr_99_1_0[t,:,:], stencil)
              #num=num+np.amax(cluster_labels)
              dummy.extend(calc_cluster_size(Pr_99_1_0[t,:,:], cluster_labels, normalize=False))
            cluster_size_all[a].extend(dummy)
            
            
# WRITE PICKLE

# Open a file and use dump()
with open('../data/cluster_size_precip_gt3mm_h_Cycle2_vs_3.pickle', 'wb') as file:
      
    # A new file will be created
    pickle.dump([cluster_size_GPM_all,cluster_size_all], file)           

01
02
03
04
05
06
07
08


In [ ]:
# CONTINUE HERE IF PICKLE IS USED

with open(f'../data/cluster_size_precip_gt3mm_h_Cycle2_vs_3.pickle', 'rb') as file:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    cluster_size_GPM_all,cluster_size_all = pickle.load(file)

In [ ]:
exp_list      = ["4km_fesom","ifs_cycle2_4km_tp","9km_nemo","ifs_cycle2_9km_tp","9km_fesom","ifs_cycle2_3km_tp"]
expid_list= ["hzfy","hqys","hz9o","hr2n","hz9o","hr0n"]
# FESOM family is red, NEMO family blue
color_list    = ["#ff0000","#ff0000","#0000ff","#0000ff","purple","darkorange"]
# give IFS resolution only, FESOM/NEMO don't change from 25/5km
name_list   = ["Cycle 3, 4.4km, IFS-FESOM","Cycle 2, 4.4km, IFS-FESOM","Cycle 3, 9km, IFS-NEMO","Cycle 2, 9km, IFS-NEMO","Cycle 3, 9km, IFS-FESOM","Cycle 2, 2.8km, IFS-FESOM"]
# Cycle 3 is solid, Cycle 2/1 dashed
array_linestyle= ["solid","dashed","solid","dashed","solid","dashed"]
# main simulation is thicker to stand out
array_linewidth= [2,1,1,1,1,1]

#bins=np.array([50 * i for i in range(100)])
bins=np.array([.9 *np.exp(.41*i) for i in range(100)])
resol='0.1x0.1$^{\circ}$, hourly'
     
plt.figure(figsize=(10,6))
for (a,name),color,linestyle,linewidth in zip(enumerate(name_list),color_list,array_linestyle,array_linewidth): 
    hist,bins=np.histogram(cluster_size_all[a], bins=bins) 
    avg = "%.1f" % np.mean(cluster_size_all[a])
    print(name)
    print(len(cluster_size_all[a]))
    plt.plot((bins[0:-1]+bins[1:])/2,hist*(bins[0:-1]+bins[1:])/2,label=f'{name}, avg: {avg}',color=color,linestyle=linestyle,linewidth=linewidth)
hist,bins=np.histogram(cluster_size_GPM_all, bins=bins) 
print(len(cluster_size_GPM_all))
avg = "%.1f" % np.mean(cluster_size_GPM_all)
plt.plot((bins[0:-1]+bins[1:])/2,hist*(bins[0:-1]+bins[1:])/2,label=f'GPM IMERG, avg: {avg}',color="k")   
plt.xlabel('size')
plt.ylabel('number * size')
plt.title(f'P > 3 mm/h, 8-connectivity, {resol}, tropics, 01-08/2020')
plt.xlim([1,10000])
plt.xscale('log')
plt.ylim([0,20000000])
plt.legend()     
plt.legend(frameon=False)    
#sns.despine(left=True, bottom=True)
ax = plt.gca()
#ax.xaxis.set_minor_locator(MultipleLocator(5))
plt.tight_layout()
plt.savefig(plotdir + f'cluster_size_distribution_number_times_size_8_connect_3mmph_NextGEMS_Cycle3_Cycle2_4km_9km_ICON_GPM_tropics_202001to08.pdf')       
plt.savefig(plotdir + f'cluster_size_distribution_number_times_size_8_connect_3mmph_NextGEMS_Cycle3_Cycle2_4km_9km_ICON_GPM_tropics_202001to08.png') 
